In [70]:
import pandas as pd
import numpy as np
import re
import string
import time
from datetime import datetime
from datetime import date
import os

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn import svm

In [71]:
# movieDBfileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Movies_director_data_3709_119Features.csv'
movieDBfileLocation = 'data2_Movies_WithAwards.csv'

In [72]:
movies_data = pd.read_csv(movieDBfileLocation)

In [73]:
movies_data.shape

(3708, 148)

In [74]:
movies_data.columns.values

array(['director_ids', 'director_names', 'movie_id', 'title',
       'review_count_user', 'review_count_critic', 'metascore',
       'rating_value', 'rating_count', 'release_date', 'runtime',
       'movie_year', 'opening_weekend_value', 'open_wk_cur_AUD',
       'open_wk_cur_CAD', 'open_wk_cur_DEM', 'open_wk_cur_GBP',
       'open_wk_cur_ITL', 'open_wk_cur_JPY', 'open_wk_cur_SEK',
       'open_wk_cur_SGD', 'open_wk_cur_USD', 'gross_value',
       'gross_cur_USD', 'budget_value', 'budget_cur_AUD',
       'budget_cur_CAD', 'budget_cur_EUR', 'budget_cur_GBP',
       'budget_cur_INR', 'budget_cur_USD', 'color_Black_and_White',
       'color_Color', 'country_Argentina', 'country_Australia',
       'country_Brazil', 'country_Canada', 'country_Denmark',
       'country_Finland', 'country_France', 'country_Germany',
       'country_Greece', 'country_Hong_Kong', 'country_India',
       'country_Italy', 'country_Japan', 'country_Mexico',
       'country_Netherlands', 'country_Other', 'country_R

In [75]:
# Uncomment in case you want to drop the one hot encoded features of awards features
dropColumnsCountry = [col for col in movies_data if col.startswith('country_')]
dropColumnsOpeningWeekend = [col for col in movies_data if col.startswith('open_wk_')]
dropColumnsBudget = [col for col in movies_data if col.startswith('budget_cur')]
dropColumnsBudget.append('gross_cur_USD')

In [76]:
movies_data[movies_data['country_USA'] == 1].count()

director_ids                           3019
director_names                         3019
movie_id                               3019
title                                  3019
review_count_user                      3019
                                       ... 
Winner_Directing                       3019
Winner_Directing (Comedy Picture)      3019
Winner_Directing (Dramatic Picture)    3019
awards_nominated                       3019
awards_won                             3019
Length: 148, dtype: int64

In [77]:
print("\n Movie dataframe Shape before drop : ",str(movies_data.shape))
movies_data = movies_data[movies_data['country_USA'] == 1]
movies_data.drop(columns = dropColumnsCountry ,inplace = True)
print("\n Movie dataframe Shape after drop : ",str(movies_data.shape))


 Movie dataframe Shape before drop :  (3708, 148)

 Movie dataframe Shape after drop :  (3019, 124)


In [78]:
movies_data[movies_data['open_wk_cur_USD'] == 1].count()

director_ids                           2679
director_names                         2679
movie_id                               2679
title                                  2679
review_count_user                      2679
                                       ... 
Winner_Directing                       2679
Winner_Directing (Comedy Picture)      2679
Winner_Directing (Dramatic Picture)    2679
awards_nominated                       2679
awards_won                             2679
Length: 124, dtype: int64

In [79]:
print("\n Movie dataframe Shape before drop : ",str(movies_data.shape))
movies_data = movies_data[movies_data['open_wk_cur_USD'] == 1]
movies_data.drop(columns = dropColumnsOpeningWeekend ,inplace = True)
print("\n Movie dataframe Shape after drop : ",str(movies_data.shape))


 Movie dataframe Shape before drop :  (3019, 124)

 Movie dataframe Shape after drop :  (2679, 115)


In [80]:
movies_data[movies_data['budget_cur_USD'] == 1].count()

director_ids                           2653
director_names                         2653
movie_id                               2653
title                                  2653
review_count_user                      2653
                                       ... 
Winner_Directing                       2653
Winner_Directing (Comedy Picture)      2653
Winner_Directing (Dramatic Picture)    2653
awards_nominated                       2653
awards_won                             2653
Length: 115, dtype: int64

In [81]:
print("\n Movie dataframe Shape before drop : ",str(movies_data.shape))
movies_data = movies_data[movies_data['budget_cur_USD'] == 1]
movies_data.drop(columns = dropColumnsBudget,inplace = True)
print("\n Movie dataframe Shape after drop : ",str(movies_data.shape))


 Movie dataframe Shape before drop :  (2679, 115)

 Movie dataframe Shape after drop :  (2653, 108)


In [82]:
USA_movie_data = movies_data.copy()
USA_movie_data.head()

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,release_date,...,Winner_Actor in a Supporting Role,Winner_Actress,Winner_Actress in a Leading Role,Winner_Actress in a Supporting Role,Winner_Assistant Director,Winner_Directing,Winner_Directing (Comedy Picture),Winner_Directing (Dramatic Picture),awards_nominated,awards_won
1,nm0000122,Charles Chaplin,tt0027977,Les temps modernes (1936) - IMDb,213,119,96,8.6,147990,24 September 1936 (France) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,nm0281808,Victor Fleming,tt0032138,"Óz, a csodák csodája (1939) - IMDb",540,213,100,8.1,299644,21 March 1940 (Hungary) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm0002030,George Cukor,tt0032138,"Óz, a csodák csodája (1939) - IMDb",540,213,100,8.1,299644,21 March 1940 (Hungary) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nm0003506,James Mangold,tt0035423,Kate & Leopold (2001) - IMDb,317,124,44,6.4,65765,25 December 2001 (USA) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,nm0039107,Ray Ashley,tt0046004,A kis szökevény (1953) - IMDb,29,45,91,7.7,1730,16 December 1953 (France) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
USA_movie_data.columns.values

array(['director_ids', 'director_names', 'movie_id', 'title',
       'review_count_user', 'review_count_critic', 'metascore',
       'rating_value', 'rating_count', 'release_date', 'runtime',
       'movie_year', 'opening_weekend_value', 'gross_value',
       'budget_value', 'color_Black_and_White', 'color_Color',
       'language_Arabic', 'language_Dutch', 'language_English',
       'language_French', 'language_German', 'language_Greek',
       'language_Hindi', 'language_Italian', 'language_Japanese',
       'language_Mandarin', 'language_Other', 'language_Portuguese',
       'language_Russian', 'language_Spanish', 'language_Swedish',
       'language_Turkish', 'genre_Action', 'genre_Adult',
       'genre_Adventure', 'genre_Animation', 'genre_Biography',
       'genre_Comedy', 'genre_Crime', 'genre_Documentary', 'genre_Drama',
       'genre_Family', 'genre_Fantasy', 'genre_Film_Noir',
       'genre_Game_Show', 'genre_History', 'genre_Horror', 'genre_Music',
       'genre_Musical', 'g

In [84]:
movies_data[['movie_id','title','movie_year','gross_value','budget_value','opening_weekend_value']]

,movie_id,title,movie_year,gross_value,budget_value,opening_weekend_value
1,tt0027977,Les temps modernes (1936) - IMDb,1936,163245.0,1500000.0,11507.0
2,tt0032138,"Óz, a csodák csodája (1939) - IMDb",1939,22202612.0,2800000.0,5354311.0
3,tt0032138,"Óz, a csodák csodája (1939) - IMDb",1939,22202612.0,2800000.0,5354311.0
4,tt0035423,Kate & Leopold (2001) - IMDb,2001,47095453.0,48000000.0,2562448.0
5,tt0046004,A kis szökevény (1953) - IMDb,1953,27373.0,30000.0,9040.0
...,...,...,...,...,...,...
3699,tt4824302,Joyeuse Fête des Mères (2016) - IMDb,2016,32463917.0,25000000.0,8369184.0
3701,tt4919484,Little Men (2016) - IMDb,2016,701836.0,2000000.0,30368.0
3703,tt4975722,Moonlight (2016) - IMDb,2016,9826174.0,5000000.0,402075.0
3704,tt5001130,Time to Choose (2015) - IMDb,2015,29233.0,3500000.0,12000.0


In [86]:
inflation_rate = 3.22

In [87]:
movies_data['gross_value'] = movies_data['gross_value'] * (1 + (inflation_rate/100)) ** (2017 - movies_data['movie_year'])
movies_data['budget_value'] = movies_data['budget_value'] * (1 + (inflation_rate/100)) ** (2017 - movies_data['movie_year'])
movies_data['opening_weekend_value'] = movies_data['opening_weekend_value'] * (1 + (inflation_rate/100)) ** (2017 - movies_data['movie_year'])


In [90]:
movies_data[['movie_id','title','movie_year','gross_value','budget_value','opening_weekend_value']]

,movie_id,title,movie_year,gross_value,budget_value,opening_weekend_value
1,tt0027977,Les temps modernes (1936) - IMDb,1936,2.126729e+06,1.954175e+07,1.499113e+05
2,tt0032138,"Óz, a csodák csodája (1939) - IMDb",1939,2.630176e+08,3.316948e+07,6.342847e+07
3,tt0032138,"Óz, a csodák csodája (1939) - IMDb",1939,2.630176e+08,3.316948e+07,6.342847e+07
4,tt0035423,Kate & Leopold (2001) - IMDb,2001,7.819890e+07,7.970084e+07,4.254776e+06
5,tt0046004,A kis szökevény (1953) - IMDb,1953,2.080698e+05,2.280384e+05,6.871557e+04
...,...,...,...,...,...,...
3699,tt4824302,Joyeuse Fête des Mères (2016) - IMDb,2016,3.350926e+07,2.580500e+07,8.638672e+06
3701,tt4919484,Little Men (2016) - IMDb,2016,7.244351e+05,2.064400e+06,3.134585e+04
3703,tt4975722,Moonlight (2016) - IMDb,2016,1.014258e+07,5.161000e+06,4.150218e+05
3704,tt5001130,Time to Choose (2015) - IMDb,2015,3.114592e+04,3.729029e+06,1.278524e+04


In [91]:
movies_data.to_csv('data3_USA_Movies_WithAwards_15thMay.csv', index = False ,encoding="utf-8-sig" )

In [16]:
break

SyntaxError: 'break' outside loop (<ipython-input-16-6aaf1f276005>, line 4)